## Setup

In [1]:
import io
import re
import string
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

### Parameters

In [2]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

### Load Stopwords

In [3]:
stopwords = []
with open('../../data/meta/stopwords','r') as f:
    for line in f:
        stopwords.append(line.rstrip())

### Generate training data

In [4]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):

    #Init
    targets, contexts, labels = [], [], []
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

    #Iterate over all sequences (sentences) in the dataset.
    for sequence in tqdm(sequences):

        #Generate positive skip-gram pairs for a sequence (sentence).
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
              sequence,
              vocabulary_size=vocab_size,
              sampling_table=sampling_table,
              window_size=window_size,
              negative_samples=0)

        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes=context_class,
                num_true=1,
                num_sampled=num_ns,
                unique=True,
                range_max=vocab_size,
                seed=seed,
                name="negative_sampling")
            
        #Build context and label vectors (for one target word)
        context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
        label = tf.constant([1] + [0]*num_ns, dtype="int64")

        #Append each element from the training example to global lists.
        targets.append(target_word)
        contexts.append(context)
        labels.append(label)

    return targets, contexts, labels

## Prepare training data for word2vec

In [175]:
text_ds = tf.data.TextLineDataset('../../data/books/frankenstein.txt')
text_ds = text_ds.filter(lambda x: tf.cast(tf.strings.length(x), bool))

### Vectorize sentences from the corpus

In [176]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,'[%s]' % re.escape(string.punctuation), '')


#Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

### Call TextVectorization.adapt on the text dataset to create vocabulary.

In [177]:
vectorize_layer.adapt(text_ds.batch(1024))

2023-01-31 18:03:04.986888: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


### Have a look at vocab

In [178]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'i', 'of', 'to', 'my', 'a', 'in', 'was', 'that', 'me', 'had', 'but', 'with', 'he', 'which', 'you', 'his']


### Vectorize and obtain sequences from the dataset

In [194]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
sequences = list(text_vector_ds.as_numpy_iterator())

### Generate training examples from sequences

In [182]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████████████████████████████████| 6389/6389 [00:13<00:00, 464.86it/s]


targets.shape: (6389,)
contexts.shape: (6389, 5)
labels.shape: (6389, 5)


### Configure the dataset for performance

In [183]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


## Model and training

In [185]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    if len(target.shape) == 2:
        target = tf.squeeze(target, axis=1)
    word_emb = self.target_embedding(target)
    context_emb = self.context_embedding(context)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    return dots

### Define loss function and compile model


In [186]:
def custom_loss(x_logit, y_true):
    return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

### Compile Model

In [199]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

### Fit

In [200]:
word2vec.fit(dataset, epochs=100)

Epoch 1/100
6/6 [==============================] - 0s 22ms/step - loss: 1.6077 - accuracy: 0.2642
Epoch 2/100
1/6 [====>.........................] - ETA: 0s - loss: 1.5952 - accuracy: 0.7227

2023-01-31 18:18:33.007800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 13ms/step - loss: 1.5962 - accuracy: 0.6610
Epoch 3/100
6/6 [==============================] - 0s 13ms/step - loss: 1.5846 - accuracy: 0.8555
Epoch 4/100
6/6 [==============================] - 0s 13ms/step - loss: 1.5718 - accuracy: 0.9312
Epoch 5/100
6/6 [==============================] - 0s 13ms/step - loss: 1.5569 - accuracy: 0.9621
Epoch 6/100
6/6 [==============================] - 0s 13ms/step - loss: 1.5394 - accuracy: 0.9740
Epoch 7/100
6/6 [==============================] - 0s 13ms/step - loss: 1.5187 - accuracy: 0.9785
Epoch 8/100
6/6 [==============================] - 0s 13ms/step - loss: 1.4943 - accuracy: 0.9818
Epoch 9/100
6/6 [==============================] - 0s 13ms/step - loss: 1.4658 - accuracy: 0.9832
Epoch 10/100
6/6 [==============================] - 0s 13ms/step - loss: 1.4328 - accuracy: 0.9832
Epoch 11/100
6/6 [==============================] - 0s 14ms/step - loss: 1.3952 - accuracy: 0.9831
Epoch 12/100
6/6 [============

Epoch 85/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0542 - accuracy: 0.9989
Epoch 86/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0535 - accuracy: 0.9989
Epoch 87/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0529 - accuracy: 0.9989
Epoch 88/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0522 - accuracy: 0.9989
Epoch 89/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0516 - accuracy: 0.9990
Epoch 90/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0510 - accuracy: 0.9990
Epoch 91/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0504 - accuracy: 0.9990
Epoch 92/100
6/6 [==============================] - 0s 14ms/step - loss: 0.0499 - accuracy: 0.9990
Epoch 93/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0493 - accuracy: 0.9990
Epoch 94/100
6/6 [==============================] - 0s 13ms/step - loss: 0.0488 - accuracy: 0.9990
Epoch 95/1

<!-- <img class="tfo-display-only-on-site" src="images/word2vec_tensorboard.png"/> -->

### Create and save the vectors and metadata files:

In [201]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')
for index, word in enumerate(vocab):
    if index == 0:
        continue  
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()

### Load the vectors and metadata back and visualize